# Plot the Near-Inertial Activity Forecast

To see how we got here, check the notebooks about [loading the data](./01_download_GFS_data.html), and about [running the slab-ocean model](./02_run_slab_model.html).

Details: https://willirath.github.io/nia-prediction-low-latitutdes

## Parameters

In [ ]:
# parameters

GFS_zarr_store = "tmp_GFS.zarr"
slab_zarr_store = "tmp_slab.zarr"

dask_kwargs = {"n_workers": 1, "threads_per_worker": 4, "memory_limit": 6e9}

buoy_locations = {
    "20N,38W": {"lat": 20.0, "lon": -38.0},
    "15N,38W": {"lat": 15.0, "lon": -38.0},
    "20.5N,23W": {"lat": 20.5, "lon": -23.0},
    "11.5N,23W": {"lat": 11.5, "lon": -23.0},
    "6S,10W": {"lat": -6.0, "lon": -10.0},
    "10S,10W": {"lat": -10.0 ,"lon": -10.0},
}

## Tech preamble

Import modules and spin up Dask cluster.

In [ ]:
import numpy as np
import xarray as xr
from dask.distributed import Client
import hvplot.xarray
import cmocean
import pandas as pd

import holoviews as hv

In [ ]:
client = Client(**dask_kwargs)
client

## Load the data

In [ ]:
ds_GFS = xr.open_zarr(GFS_zarr_store)
ds_slab = xr.open_zarr(slab_zarr_store)

## Find start of forecast period

In [ ]:
start_of_forecast = (~ds_GFS["is_forecast"].astype(bool)).sum().compute().data
start_of_forecast = ds_GFS["time"].data[start_of_forecast]
print(start_of_forecast)

## Plot for buoy locations

In [ ]:
for name, location in buoy_locations.items():
    buoy_ds = ds_slab.sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    buoy_ds["taux"] = ds_GFS["taux"].sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    buoy_ds["tauy"] = ds_GFS["tauy"].sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    
    display(
        (
            buoy_ds["taux"].hvplot.line(label="taux", width=600, height=150)
            * buoy_ds["tauy"].hvplot.line(label="tauy")
            + buoy_ds["u_slab"].hvplot.line(label="u", width=600, height=150)
            * buoy_ds["v_slab"].hvplot.line(label="v")
            * buoy_ds["umag_slab"].hvplot.line(label="umag")
            * hv.VLine(start_of_forecast)
        ).cols(1).opts(title=name)
    )

## Plot overview

In [ ]:
hvplot.xarray.contourf?

In [ ]:
slab_umag_max = ds_slab["umag_slab"].where(ds_GFS["is_forecast"]).max("time")
slab_umag_max.hvplot.contourf(
    levels=list(np.linspace(0, 3, 41)),
    cmap=cmocean.cm.speed,
    width=800, height=500
)

In [ ]:
SLP_mean_analysis = ds_GFS["SLP"].where(~ds_GFS["is_forecast"]).mean("time")
SLP_anomaly = ds_GFS["SLP"] - SLP_mean_analysis

In [ ]:
display(
    SLP_mean_analysis.hvplot.contourf(
        levels=list(np.linspace(97_000, 104_000, 51)),
        cmap=cmocean.cm.thermal,
        width=800, height=300
    )
)

In [ ]:
plot_time = [start_of_forecast + np.timedelta64(int(24 * 3600 * 1e9) * n, "ns") for n in range(0, 4)]

display(
    (
        SLP_anomaly.sel(time=plot_time[0]).hvplot.contourf(
            levels=list(np.linspace(-1000, 1000, 51)),
            cmap=cmocean.cm.balance,
            width=800, height=300,
            label=f"SLP anomaly, {plot_time[0]}"
        )
        + SLP_anomaly.sel(time=plot_time[1]).hvplot.contourf(
            levels=list(np.linspace(-1000, 1000, 51)),
            cmap=cmocean.cm.balance,
            width=800, height=300,
            label=f"SLP anomaly, {plot_time[1]}"
        )
        + SLP_anomaly.sel(time=plot_time[2]).hvplot.contourf(
            levels=list(np.linspace(-1000, 1000, 51)),
            cmap=cmocean.cm.balance,
            width=800, height=300,
            label=f"SLP anomaly, {plot_time[2]}"
        )
        + SLP_anomaly.sel(time=plot_time[3]).hvplot.contourf(
            levels=list(np.linspace(-1000, 1000, 51)),
            cmap=cmocean.cm.balance,
            width=800, height=300,
            label=f"SLP anomaly, {plot_time[3]}"
        )
    ).cols(1)
)

---

In [ ]:
!echo "Finished: $(date -Ins)"

---
See https://github.com/willirath/nia-prediction-low-latitutdes for details.